### Fake News Detection Problem

In [5]:
import pandas as pd
import numpy as np
import re
import string


In [6]:
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

### Reading Fake News Dataset

In [7]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


### Reading True News Dataset

In [8]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [9]:
df_fake["class"] = 0
df_true["class"] = 1
df_true.head()


,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


### Checking Shape for each Dataset // no of rows vs col

In [10]:
df_fake.shape, df_true.shape

((23481, 5), (21417, 5))

### verification

In [11]:
df_fake_manual_testing = df_fake.tail(10)
for i in range(23480, 23470, -1):
    df_fake.drop([i], axis=0, inplace=True)

df_true_manual_testing = df_true.tail(10)
for i in range(21416, 21406, -1):
    df_true.drop([i], axis=0, inplace=True)

In [12]:
df_manual_testing = pd.concat([df_fake_manual_testing, df_true_manual_testing], axis=0)
# saving file as csc formate
df_manual_testing.to_csv("manual_testing.csv")

In [13]:
df_combined = pd.concat([df_fake, df_true], axis = 0)
df_combined.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [14]:
df_final = df_combined.drop(["title", "subject", "date"], axis = 1)
df_final.head()

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [15]:
df_final = df_final.sample(frac=1).reset_index(drop=True)
df_final.head()

,text,class
0,UPDATE: ANTHONY WEINER JUST DELETED HIS TWITTE...,0
1,Bristol Palin didn t go to Harvard Law School ...,0
2,"GOMA, Democratic Republic of Congo (Reuters) -...",1
3,There definitely seems to be a new sense of en...,0
4,I don t really know about that Nice try Hi...,0


In [16]:
df_final.isnull().sum()

text     0
class    0
dtype: int64

### Text Processing

In [17]:
!pip install nltk
!pip install textblob
!pip install pyspellchecker

In [18]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import re

In [19]:
port_stem = PorterStemmer()
port_stem

<PorterStemmer>

In [20]:
port_stem.stem("HELLO yhoo")

'hello yhoo'

In [21]:
def txt_processing(text):
  txt = re.sub('[^a-zA-Z]', ' ', text)
  txt = txt.lower()
  txt = txt.split()
  txt = [port_stem.stem(word) for word in txt if not word in stopwords.words('english')]
  txt = ' '.join(txt)
  return txt

In [25]:
txt_processing('HELLO yhoo the dog is a *@#%')

'hello yhoo dog'

In [23]:
df_final['text'] = df_final['text'].apply(txt_processing)
df_final.head()

,text,class
0,updat anthoni weiner delet twitter account pho...,0
1,bristol palin go harvard law school like presi...,0
2,goma democrat republ congo reuter rebel spokes...,1
3,definit seem new sens entitl among donald trum...,0
4,realli know nice tri hillari,0
